In [ ]:
# Import the class from the Python file (module)
import pandas as pd
import matplotlib.pyplot as plt
import os
# from dotenv import load_dotenv
# from pathlib import Path
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from BinanceClient import BinanceClient
import numpy as np
from typing import Final
import joblib
from BatchFeatures import BatchFeatures
from datetime import datetime, timedelta
%matplotlib widget

In [2]:
import os
from datetime import datetime, timedelta, timezone

# ----------------------------
# Config
# ----------------------------
pair = "ETHUSDT"
interval = "5m"
weeks = 52  # duration in weeks

api_secret = os.getenv("BINANCE_SECRET_KEY")
api_key = os.getenv("BINANCE_API_KEY")
# if not api_key or not api_secret:
#     raise RuntimeError("Missing BINANCE_API_KEY / BINANCE_SECRET_KEY in environment variables.")

# ----------------------------
# Helpers
# ----------------------------
def interval_slug(s: str) -> str:
    # Keep it file-system friendly
    return s.strip().replace(" ", "").replace("/", "").lower()

def make_db_name(pair: str, interval: str, weeks: int) -> str:
    return f"{pair}_{interval_slug(interval)}_{weeks}weeks.db"

db_name = make_db_name(pair, interval, weeks)
db_path = db_name  # change to os.path.join("data", db_name) if you store DBs in a folder

print(f"DB: {db_path}")

# ----------------------------
# Client init
# ----------------------------
binance_client = BinanceClient(db_path)
binance_client.set_interval(interval)

# ----------------------------
# Load or fetch
# ----------------------------
df = None

# If DB file exists, try loading data for this pair
if os.path.exists(db_path):
    df = binance_client.fetch_data_from_db(pair)
    if df is not None and not df.empty:
        print(f"Loaded {len(df):,} rows from DB.")
    else:
        df = None  # treat as missing

# If DB missing or empty -> fetch, store, then load
if df is None:
    print("No usable DB data found -> fetching from Binance...")

    # connect to Binance only if we need to fetch
    binance_client.make(api_key, api_secret)

    server_time = binance_client.get_server_time()
    end_dt = datetime.fromtimestamp(server_time["serverTime"] / 1000, tz=timezone.utc)
    start_dt = end_dt - timedelta(weeks=weeks)

    start_ms = int(start_dt.timestamp() * 1000)
    end_ms = int(end_dt.timestamp() * 1000)

    data = binance_client.fetch_data(pair, start_ms, end_ms)
    if data is None or data.empty:
        raise RuntimeError("No data returned from Binance for the requested range.")

    binance_client.store_data_to_db(pair, data)

    # Always load final df from DB so it is consistent
    df = binance_client.fetch_data_from_db(pair)
    if df is None or df.empty:
        raise RuntimeError("Data was fetched/stored but DB load returned empty. Check DB write/read logic.")

    print(f"Fetched + stored + loaded {len(df):,} rows.")

# Final sanity
df = df.sort_index()
print(df.head())
print(df.tail())


DB: ETHUSDT_5m_52weeks.db
Loaded 104,832 rows from DB.
                        open     high      low    close     volume
timestamp                                                         
2025-01-25 16:30:00  3343.44  3344.95  3335.69  3339.98   906.9382
2025-01-25 16:35:00  3339.99  3343.27  3338.56  3342.23   860.8281
2025-01-25 16:40:00  3342.23  3343.00  3334.62  3336.28   713.0242
2025-01-25 16:45:00  3336.28  3340.62  3332.00  3334.07  1102.9760
2025-01-25 16:50:00  3334.10  3338.20  3333.60  3335.98   525.4775
                        open     high      low    close    volume
timestamp                                                        
2026-01-24 16:05:00  2956.45  2961.13  2956.45  2960.42  139.2611
2026-01-24 16:10:00  2960.42  2961.62  2955.54  2956.86  518.5347
2026-01-24 16:15:00  2956.86  2958.15  2952.23  2954.01  628.7756
2026-01-24 16:20:00  2954.00  2955.39  2953.76  2953.76  138.2384
2026-01-24 16:25:00  2953.76  2953.77  2952.94  2952.95   29.1532


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Core ---
INTERVAL = "5m"
ORDER_TTL = 6         # order valid for 6 candles = 30 min (try 6, 12)
HOLD_BARS = 6         # exit 6 candles after fill (30 min) (try 6, 12, 18)
K_ATR = 1.0           # entry offset: 1.0 * ATR% (try 0.6, 0.8, 1.0, 1.2)

# --- Fees (BNB discount) ---
FEE_PER_SIDE = 0.00075  # 0.075%
ROUND_TRIP_FEE = 2 * FEE_PER_SIDE


In [4]:
df_m = df.copy().sort_index()

# ATR proxy as % of price (range-based). Keep it simple and robust.
df_m["atrp_14"] = ((df_m["high"] - df_m["low"]) / df_m["close"]).rolling(14).mean()

# basic return features (optional, for later ML gating)
df_m["ret_1"] = df_m["close"].pct_change(1)
df_m["vol_12"] = df_m["ret_1"].rolling(12).std()

df_m = df_m.dropna().copy()


In [5]:
def maker_backtest(df_in: pd.DataFrame,
                   k_atr: float,
                   order_ttl: int,
                   hold_bars: int,
                   round_trip_fee: float):
    """
    Maker strategy:
      - Place limit buy at close*(1 - k_atr*atrp)
      - If low touches within order_ttl bars => filled at limit price
      - Exit at close[fill_idx + hold_bars]
      - One active order/trade at a time (non-overlapping)
    """
    dfb = df_in.copy()
    n = len(dfb)
    idx = dfb.index

    # precompute entry prices per bar
    dfb["entry_price"] = dfb["close"] * (1.0 - k_atr * dfb["atrp_14"])

    fills = 0
    placed = 0

    trade_returns = []
    trade_entry_ts = []
    trade_fill_ts = []
    trade_exit_ts = []
    trade_entry_px = []
    trade_exit_px = []

    i = 0
    while i < n - (order_ttl + hold_bars):
        placed += 1
        entry_px = float(dfb["entry_price"].iat[i])

        # look ahead for fill within ttl
        fill_j = None
        j_end = i + order_ttl
        lows = dfb["low"].iloc[i+1:j_end+1]  # next bars only
        hit = lows <= entry_px
        if hit.any():
            # first bar where low <= entry_px
            fill_pos = int(np.argmax(hit.values))  # index within the slice
            fill_j = (i + 1) + fill_pos

        if fill_j is None:
            # order expired; move to next bar after ttl
            i = j_end + 1
            continue

        fills += 1
        exit_j = fill_j + hold_bars

        exit_px = float(dfb["close"].iat[exit_j])

        gross = (exit_px / entry_px) - 1.0
        net = gross - round_trip_fee

        trade_returns.append(net)
        trade_entry_ts.append(idx[i])
        trade_fill_ts.append(idx[fill_j])
        trade_exit_ts.append(idx[exit_j])
        trade_entry_px.append(entry_px)
        trade_exit_px.append(exit_px)

        # jump to after exit (non-overlapping)
        i = exit_j + 1

    trade_returns = np.array(trade_returns, dtype=float)
    equity = np.cumprod(1.0 + trade_returns) if len(trade_returns) else np.array([1.0])

    result = {
        "placed": placed,
        "filled": fills,
        "fill_rate": (fills / placed) if placed else 0.0,
        "trades": len(trade_returns),
        "avg_trade": (trade_returns.mean() if len(trade_returns) else 0.0),
        "win_rate": ((trade_returns > 0).mean() if len(trade_returns) else 0.0),
        "total_return": equity[-1] - 1.0,
        "equity": equity,
        "trades_df": pd.DataFrame({
            "entry_time": trade_entry_ts,
            "fill_time": trade_fill_ts,
            "exit_time": trade_exit_ts,
            "entry_px": trade_entry_px,
            "exit_px": trade_exit_px,
            "net_return": trade_returns
        })
    }
    return result


res = maker_backtest(
    df_in=df_m,
    k_atr=K_ATR,
    order_ttl=ORDER_TTL,
    hold_bars=HOLD_BARS,
    round_trip_fee=ROUND_TRIP_FEE
)

print(f"K_ATR={K_ATR}, ORDER_TTL={ORDER_TTL}, HOLD_BARS={HOLD_BARS}, ROUND_TRIP_FEE={ROUND_TRIP_FEE:.4%}")
print(f"Orders placed: {res['placed']}")
print(f"Fills: {res['filled']} (fill rate {res['fill_rate']:.2%})")
print(f"Trades: {res['trades']}")
print(f"Avg trade (net): {res['avg_trade']:.4%}")
print(f"Win rate: {res['win_rate']:.2%}")
print(f"Total return (compounded): {res['total_return']:.2%}")

# plt.figure()
# plt.plot(res["equity"])
# plt.title("Maker Strategy Equity (Per-Trade, Non-overlapping)")
# plt.xlabel("Trade #")
# plt.ylabel("Equity")
# plt.show()


K_ATR=1.0, ORDER_TTL=6, HOLD_BARS=6, ROUND_TRIP_FEE=0.1500%
Orders placed: 12449
Fills: 6335 (fill rate 50.89%)
Trades: 6335
Avg trade (net): -0.1541%
Win rate: 35.15%
Total return (compounded): -99.99%


In [81]:
Ks = [0.4, 0.6, 0.8, 1.0, 1.2, 1.5]
for k in Ks:
    r = maker_backtest(df_m, k_atr=k, order_ttl=ORDER_TTL, hold_bars=HOLD_BARS, round_trip_fee=ROUND_TRIP_FEE)
    print(f"K_ATR={k:>3} | trades={r['trades']:>4} | fill={r['fill_rate']*100:>5.1f}% | avg={r['avg_trade']*100:>7.3f}% | total={r['total_return']*100:>7.2f}%")


K_ATR=0.4 | trades=9655 | fill= 77.5% | avg= -0.162% | total=-100.00%
K_ATR=0.6 | trades=8511 | fill= 68.8% | avg= -0.162% | total=-100.00%
K_ATR=0.8 | trades=7487 | fill= 60.7% | avg= -0.162% | total=-100.00%
K_ATR=1.0 | trades=6658 | fill= 53.8% | avg= -0.155% | total=-100.00%
K_ATR=1.2 | trades=5817 | fill= 46.6% | avg= -0.151% | total= -99.99%
K_ATR=1.5 | trades=4702 | fill= 36.9% | avg= -0.156% | total= -99.94%
